<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/2023notebooks/2023_0113lam_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ここはお遊びなので，スキップしても良い
import IPython
#IPython.display.Image(url="https://livedoor.blogimg.jp/ftb001/imgs/b/4/b4629a79.jpg")
IPython.display.Image(url="https://uy-allstars.com/_assets/images/pages/char/detail/webp/lum@pc.webp")

In [ ]:
%config InlineBackend.figure_format = 'retina'
try:
    import bit
except ImportError:
    !pip install ipynbname --upgrade > /dev/null 2>&1 
    !git clone https://github.com/ShinAsakawa/bit.git
import bit

isColab = bit.isColab
HOME = bit.HOME

if isColab:
    !apt install aptitude
    !aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
    !pip install mecab-python3==0.7
    !pip install jaconv
    
    import MeCab
    wakati = MeCab.Tagger('-Owakati').parse
    yomi = MeCab.Tagger('-Oyomi').parse
else:
    from ccap.mecab_settings import yomi
    from ccap.mecab_settings import wakati

# 自作ライブラリ LAM の読み込み
if isColab:
    !git clone https://github.com/ShinAsakawa/ccap.git
    !git clone https://github.com/ShinAsakawa/lam.git

# 1 準備作業



## 1.1 ライブラリのインポート

1.   直下セルは，mecab をコンパイルするので時間がかかるので注意



## 1.2 Colab 上で実行する場合，必要なファイルのアップロード

In [ ]:
# upload `pslex71utf8.txt`, NTT 日本語の語彙特性 頻度データ
# upload `lam/2022_0508SALA_TLPA.json` SALA と TLPA のデータが必要になるかもしれない。
if isColab:
    from google.colab import files
    uploaded = files.upload()

## 1.3 パラメータ設定

In [ ]:
%reload_ext autoreload
%autoreload 2

import torch
import lam
device = lam.device  # CPU or GPU の選択

from termcolor import colored

# シミュレーションに必要なパラメータの設定
params = {
    'traindata_size':  20000,   # 訓練データ数，NTT 日本語語彙特性の高頻度語を上位から抽出
    #'traindata_size': 301612,  # 訓練データ数，NTT 日本語語彙特性の高頻度語を上位から抽出
    'epochs': 20,               # 学習のためのエポック数
    'hidden_size': 128,         # 中間層のニューロン数
    'random_seed': 42,          # 乱数の種。ダグラス・アダムス著「銀河ヒッチハイカーズガイド」

    # 以下 `source` と `target` を定義することで，別の課題を実行可能
    'source': 'orth',          # ['orth', 'phon', 'mora', 'mora_p', 'mora_p_r']
    'target': 'phon',         # ['orth', 'phon', 'mora', 'mora_p', 'mora_p_r']
    #'target': 'mora_p_r',      # ['orth', 'phon', 'mora', 'mora_p', 'mora_p_r']
    # 'orthography': 書記素, 
    # 'phonology': 音韻, 
    # 'mora': モーラ
    # 'mora_p': モーラを silius による音分解
    # 'mora_p_r': モーラの silius 音分解の逆
    'pretrained': False,          # True であれば訓練済ファイルを読み込む
    #'pretrained': True,          # True であれば訓練済ファイルを読み込む
    #'isTrain'   : True,          # True であれば学習する
    
    # 学習済のモデルパラメータを保存するファイル名
    #'path_saved': '2022_0607lam_o2p_hid32_vocab10k.pt', 
    #'path_saved': '2022_0829lam_p2p_hid24_vocab10k.pt',
    'path_saved': False,                      # 保存しない場合
    
    # 結果の散布図を保存するファイル名    
    #'path_graph': '2022_0829lam_p2p_hid24_vocab10k.pdf',
    'path_graph': False,                     # 保存しない場合

    'lr': 0.0001,                              # 学習率
    'dropout_p': 0.0,                         # ドロップアウト率
    'teacher_forcing_ratio': 0.5,             # 教師強制を行う確率
    'optim_func': torch.optim.Adam,           # 最適化アルゴリズム ['torch.optim.Adam', 'torch.optim.SGD', 'torch.optim.AdamW']
    'loss_func' :torch.nn.CrossEntropyLoss(), # 交差エントロピー損失 ['torch.nn.NLLLoss()', or 'torch.nn.CrossEntropyLoss()']
}

## 1.3 データセットの設定

In [ ]:
_vocab = lam.VOCAB(traindata_size=params['traindata_size'], 
                   w2v=None, 
                   yomi=yomi) 

source = params['source']
target = params['target']

# _max_len はアテンション機構のデコーダで必要になるため，全条件で最長の長さを指定する必要がある
_max_len = _vocab.max_ortho_length
_max_len = _max_len if _max_len > _vocab.max_phone_length else _vocab.max_phone_length
_max_len = _max_len if _max_len > _vocab.max_mora_length else _vocab.max_mora_length
_max_len = _max_len if _max_len > _vocab.max_mora_p_length else _vocab.max_mora_p_length
_vocab.max_length = _max_len + 1
print(colored(f'_vocab.max_length: {_vocab.max_length}', 'blue', attrs=['bold']))

# ソース，すなわち encoder 側の，項目番号，項目 ID，decoder 側の項目，項目 ID を設定
source_vocab, source_ids, target_vocab, target_ids = lam.get_soure_and_target_from_params(
    params=None,
    _vocab=_vocab,
    source=source,
    target=target,
    is_print=True)

print(colored(f'source:{source}','blue', attrs=['bold']), f'{source_vocab}')
print(colored(f'target:{target}','cyan', attrs=['bold']), f'{target_vocab}')
print(colored(f'source_ids:{source_ids}','blue', attrs=['bold']), f'{source_ids}')
print(colored(f'target_ids:{target_ids}','cyan', attrs=['bold']), f'{target_ids}')

# 検証データとして，TLPA と SALA のデータを用いる
tlpa1, tlpa2, tlpa3, tlpa4, sala_r29, sala_r30, sala_r31 = lam.read_json_tlpa1234_sala_r29_30_31(
    json_fname='lam/2022_0508SALA_TLPA.json')

_dataset = {}
_data_names = ['tlpa2', 'tlpa3', 'tlpa4', 'sala_r29', 'sala_r30', 'sala_r31']
for data in _data_names:
    _dataset[data] = {'rawdata':eval(data),
                      'pdata': lam.make_vocab_dataset(eval(data),vocab=_vocab)}

# 以下は後から付け足したので，コードが汚くなっている。
# 時間ができたらコードの整理をすること
# X_vals = lam.make_X_vals(_dataset=_dataset,
#                          source_vocab=source_vocab,
#                          target_vocab=target_vocab,
#                          source_ids=source_ids,
#                          target_ids=target_ids
#                         )

In [ ]:
train_wordlist = [v['orig'] for k, v in _vocab.train_data.items()]
print(len(train_wordlist))

### 任意の単語 orthography を変換するための関数

In [ ]:
def _get_ids_from_orth(orth_wrd:str='てれび',
                       __vocab:lam.lam.VOCAB=_vocab):
    _yomi, _phon, _phon_r, _orth, _orth_r, _mora, _mora_r = __vocab.get7lists_from_orth(orth=orth_wrd)
    phon_ids, phon_ids_r, orth_ids, orth_ids_r, mora_ids, mora_ids_r = __vocab.get6ids(yomi=_yomi, _phon=_phon, _orth=_orth)
    
    return {'_yomi':_yomi,
            '_phon':_phon,
            '_phon_r':_phon_r,
            '_orth':_orth,
            '_orth_r':_orth_r,
            '_mora':_mora,
            '_mora_r':_mora_r,
            'phon_ids':phon_ids,
            'phon_ids_r':phon_ids_r,
            'orth_ids':orth_ids,
            'orth_ids_r':orth_ids_r,
            'mora_ids':mora_ids,
            'mora_ids_r':mora_ids_r,
           }
    
print(_get_ids_from_orth())


def orth_ids2tkn(ids:list):
    return [_vocab.ortho_vocab[idx] for idx in ids]

def orth_tkn2ids(tkn:list):
    return [_vocab.ortho_vocab.index(_tkn) if _tkn in _vocab.ortho_vocab else _vocab.ortho_vocab.index('<UNK>') for _tkn in tkn]

def mora_p_ids2tkn(ids:list):
    return [_vocab.mora_p_vocab[idx] for idx in ids]

def mora_p_tkn2ids(tkn:list):
    return [_vocab.mora_p_vocab.index(_tkn) if _tkn in _vocab.mora_p_vocab else _vocab.mora_p_vocab('<UNK>') for _tkn in tkn]

def mora_ids2tkn(ids:list):
    return [_vocab.mora_vocab[idx] for idx in ids]

def mora_tkn2ids(tkn:list):
    return [_vocab.mora_vocab.index(_tkn) if _tkn in _vocab.mora_vocab else _vocab.mora_vocab('<UNK>') for _tkn in tkn]
    #return [_vocab.mora_vocab.index(_tkn) for _tkn in tkn]


_ids = [111, 298]
print(orth_ids2tkn(_ids))
print(orth_tkn2ids(orth_ids2tkn(_ids)))

print(orth_tkn2ids('新しい'))
print(orth_tkn2ids('神経心理学'))
print(orth_ids2tkn(orth_tkn2ids('神経心理学')))
print(mora_p_ids2tkn([17,19,11,4,32,17]))
print(mora_p_tkn2ids(mora_p_ids2tkn([17,19,11,4,32,17])))
print(mora_ids2tkn([37,139,31,7]))
print(mora_tkn2ids(mora_ids2tkn([37,139,31,7])))

### SALA and TLPA dataset

In [ ]:
# 検証データとして，TLPA と SALA のデータを用いる
tlpa1, tlpa2, tlpa3, tlpa4, sala_r29, sala_r30, sala_r31 = lam.read_json_tlpa1234_sala_r29_30_31(
    json_fname='lam/2022_0508SALA_TLPA.json')

_dataset = {}
_data_names = ['tlpa2', 'tlpa3', 'tlpa4', 'sala_r29', 'sala_r30', 'sala_r31']
for data in _data_names:
    _dataset[data] = {'rawdata':eval(data),
                      'pdata': lam.make_vocab_dataset(eval(data), vocab=_vocab)}

# 以下は後から付け足したので，コードが汚くなっている。
# 時間ができたらコードの整理をすること
# X_vals = lam.make_X_vals(_dataset=_dataset,
#                          source_vocab=source_vocab,
#                          target_vocab=target_vocab,
#                          source_ids=source_ids,
#                          target_ids=target_ids)

_data_names = ['tlpa2', 'tlpa3', 'tlpa4', 'sala_r29', 'sala_r30', 'sala_r31']    
for data in _data_names:
    print(colored(data, 'blue', attrs=['bold']), eval(data))

### Fushimi1999 データセット

In [ ]:
fushimi1999 = {
    'HF___consist__': ['戦争', '倉庫', '医学', '注意', '記念', '番号', '料理', '完全', '開始', '印刷',
                       '連続', '予約', '多少', '教員', '当局', '材料', '夕刊', '労働', '運送', '電池' ], # consistent, 'high-frequency words
    'HF___inconsist': ['反対', '失敗', '作品', '指定', '実験', '決定', '独占', '独身', '固定', '食品',
                       '表明', '安定', '各種', '役所', '海岸', '決算', '地帯', '道路', '安打', '楽団' ], # inconsistent, 'high-frequency words
    'HF___atypical_': ['仲間', '夫婦', '人間', '神経', '相手', '反発', '化粧', '建物', '彼女', '毛糸', 
                       '場合', '台風', '夜間', '人形', '東西', '地元', '松原', '競馬', '大幅', '貸家' ], # inconsistent atypical, 'high-frequency words
    'LF___consist__': ['集計', '観察', '予告', '動脈', '理学', '信任', '任務', '返信', '医局', '低温', 
                       '区別', '永続', '持続', '試練', '満開', '軍備', '製材', '銀貨', '急送', '改選' ], # consistent, 'low-frequecy words
    'LF___inconsist': ['表紙', '指針', '熱帯', '作詞', '決着', '食費', '古代', '地形', '役場', '品種', 
                       '祝福', '金銭', '根底', '接種', '経由', '郷土', '街路', '宿直', '曲折', '越境' ], # inconsistent, 'low-frequency words
    'LF___atypical_': ['強引', '寿命', '豆腐', '出前', '歌声', '近道', '間口', '風物', '面影', '眼鏡', 
                       '居所', '献立', '小雨', '毛皮', '鳥居', '仲買', '頭取', '極上', '奉行', '夢路' ], # inconsistent atypical, 'low-frequncy words
    'HFNW_consist__': ['集学', '信別', '製信', '運学', '番送', '電続', '完意', '軍開', '動選', '当働', 
                       '予続', '倉理', '予少', '教池', '理任', '銀務', '連料', '開員', '注全', '記争' ], # consistent, 'high-character-frequency nonwords
    'HFNW_inconsist': ['作明', '風行', '失定', '指団', '決所', '各算', '海身', '東発', '楽験', '作代',
                       '反原', '独対', '歌上', '反定', '独定', '場家', '安種', '経着', '決土', '松合' ], # inconsistent biased, 'high-character-frequency nonwords
    'HFNW_ambiguous': ['表品', '実定', '人風', '神間', '相経', '人元', '小引', '指場', '毛所', '台手',
                       '間物', '道品', '出取', '建馬', '大婦', '地打', '化間', '面口', '金由', '彼間' ], # inconsistent ambigous, 'high-character-frequency nonwords
    'LFNW_consist__': ['急材', '戦刊', '返計', '印念', '低局', '労号', '満送', '永告', '試脈', '観備',
                       '材約', '夕局', '医庫', '任続', '医貨', '改練', '区温', '多始', '材刷', '持察' ], # consistent, 'low-character-frequency nonwords
    'LFNW_inconsist': ['食占', '表底', '宿帯', '決帯', '古費', '安敗', '役針', '近命', '眼道', '豆立',
                       '街直', '固路', '郷種', '品路', '曲銭', '献居', '奉買', '根境', '役岸', '祝折' ], # inconsistent biased, 'low-character-frequency nonwords
    'LFNW_ambiguous': ['食形', '接紙', '競物', '地詞', '強腐', '頭路', '毛西', '夜糸', '仲影', '熱福',
                       '寿前', '鳥雨', '地粧', '越種', '仲女', '極鏡', '夢皮', '居声', '貸形', '夫幅' ], # inconsistent ambigous, 'low-character-frequency nonwords
}

for k, v in fushimi1999.items():
    print(colored(k, 'blue', attrs=['bold']), v)

fushimi1999_list = []
for k, v in fushimi1999.items():
    for _v in v:
        fushimi1999_list.append(_v)

train_wordlist = [v['orig'] for k, v in _vocab.train_data.items()]

for i, wrd in enumerate(fushimi1999_list):
    if wrd in train_wordlist:
        idx = train_wordlist.index(wrd)
        print(f'{i:3d} {wrd}:{idx:5d}')
        #print(i, wrd, idx, _vocab.train_data[idx])        

In [ ]:
for i, w in enumerate(fushimi1999_list):
    ids = orth_tkn2ids(w)
    tnk = orth_ids2tkn(ids)
    if i > 125:
        print(i, w, ids, tnk)
        o = _get_ids_from_orth(orth_wrd=w)
        print(colored((i,w),'blue',attrs=['bold']), o)

### テストのための一文字の書記素データセットの作成

In [ ]:
one_chars0 = '０１２３４５６７８９ＡＢＣＤＥＦＧＨＩＪＫＬＭＮＯＰＱＲＳＴＵＶＷＸＹＺａｂｃｄｅｆｇｈｉｊｋｌｍｎｏｐｑｒｓｔｕｖｗｘｙｚ'
one_chars1 = 'あいうえおかがきぎくぐけげこごさざしじすずせぜそぞただちぢつづてでとどなにぬねのはばぱひびぴふぶぷへべぺほぼぽまみむめもやゆよらりるれろわをん'
one_chars2 = 'アイウエオカガキギクグケゲコゴサザシジスズセゼソゾタダチヂッツヅテデトドナニヌネノハバパヒビピフブプヘベペホボポマミムメモヤユヨラリルレロワヲン'
one_chars = one_chars0 + one_chars1 + one_chars2
for ch in one_chars:
    tmp = _get_ids_from_orth(ch)
    __src, __tgt, __yomi = tmp[_src], tmp[_tgt], tmp['_yomi']
    print(colored(ch, 'blue', attrs=['bold']), f'__src:{__src}, __tgt:{__tgt}, __yomi:{__yomi}')